# HW8 (20')

<font size='4'>

For this assignment, it is a combination of jupyter notebook assignment and python scripts.

For Q1, please upload your outputs including codes and graphics to your own GitHub repository. <br> You will need to disclose your GitHub repository below.

For Q2, please submit this jupyter notebook as an HTML or PDF file.

First of all, print your name (First and Last) below.

## 0. Import relevant packages

In [82]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.formula.api as smf

## Q1. Convert your HW7 to python scripts. (10')

<font size='4'>

- Under your working directory, there should be a folder called `self_py_fun`.
- Create a new python file called `HW8Fun.py` and move previously defined functions `produce_trun_mean_cov()`, `plot_trunc_mean()`, and `plot_trunc_cov()` to that file. Make sure you import proper packages.
- Create another main file `HW8_main.py`.
- Import relevant packages, modules, and/or function.
- Copy the global variables and call your functions inside `HW8_main.py`.
- A major difference compared to HW7 is that you are asked to save those figures to your local working environment.
    - Create a new directory `K114` under your current working directory.
    - For mean functions, please save it as a `Mean.png` output using `plt.savefig()` function.
    - The changes should be made within `HW8Fun.py` rather than `HW8_main.py`.
    - For covariance matrices, please save them as `Covariance_Target.png`, `Covariance_Non-Target.png`, and `Covariance_All.png` outputs using the same function above.
    - To summarize, there should be **four** figures under `K114` folder.
- Upload your entire work to your GitHub repository via push button.

In [4]:
# Provide your GitHub repository link below in the Markdown chunk. Remember to make it public and make the link clickable.
# Do not include sensitive information in your GitHub repository.
# https://github.com/sapnadavid/BIOS-584/tree/main


## Q2. A real-world data anlaysis using `Pandas` and `Scipy` (10')

<font size='4'>

- Back to the `PTSD dataset.xlsx`, let's import the dataset and name it `ptsd_df`. (no point since everyone has done it a couple of times before.)

In [21]:
# Write your own code

import pandas as pd
file_path = "/Users/sapnadavid/Documents/GitHub/BIOS-584/PTSD dataset.xlsx"

# Import the dataset
ptsd_df = pd.read_excel(file_path)


### Q2.1. Univariate comparison (3')

<font size='4'> 
    
- Suppose that we would like to examine the utility/effect of an intervention program for patients with PTSD.
- We measure PCL5 scores at completion (`pcl5week_score.completion`) and PCL5 score at 3-month follow-up (`pcl5month_score.3_month_follow_up`). Let's assume the first score is pre-intervention and the second score is post-intervention.
- Report the summary statistics for each variable including mean, std, median, Q1, and Q3.
- Note that each patient will receive such two PCL5 scores. Use a appropriate statistical test to perform the univariate comparison. Report the outputing statistic and p-value.
- Before you run the statistic test, determine the data type and check the missingness of two columns. In particular, report the number of NA values for each variable.

In [25]:
# Write your own code
ptsd_df = pd.read_excel(file_path, sheet_name="main_dataset", header=None)
print(ptsd_df.head(10))
ptsd_df.columns = ptsd_df.iloc[0].astype(str).str.strip()

ptsd_df = ptsd_df[1:].reset_index(drop=True)

# Check column names
print("Columns after setting header:")
print(ptsd_df.columns)
pre_col = 'pcl5week_score.completion'
post_col = 'pcl5month_score.3_month_follow_up'
ptsd_df[pre_col] = pd.to_numeric(ptsd_df[pre_col], errors='coerce')
ptsd_df[post_col] = pd.to_numeric(ptsd_df[post_col], errors='coerce')
na_counts = ptsd_df[[pre_col, post_col]].isna().sum()
print("Number of missing values per column:")
print(na_counts)
summary_stats = ptsd_df[[pre_col, post_col]].agg(
    ['mean', 'std', 'median',
     lambda x: np.percentile(x.dropna(), 25),  # Q1
     lambda x: np.percentile(x.dropna(), 75)]  # Q3
)
summary_stats.rename(index={'<lambda_0>': 'Q1', '<lambda_1>': 'Q3'}, inplace=True)
print("\nSummary statistics:")
print(summary_stats)
paired_data = ptsd_df[[pre_col, post_col]].dropna()

# Paired t-test
t_stat, t_pval = stats.ttest_rel(paired_data[pre_col], paired_data[post_col])
print(f"\nPaired t-test:\nt-statistic = {t_stat:.3f}, p-value = {t_pval:.3f}")

         0                 1              2                   3    \
0  record_id  ptsdpresent_caps  caps_minuspcl  caps_minuspcl_code   
1        460                 1            -56                   1   
2       5942                 1            -49                   1   
3       5366                 1            -48                   1   
4       3480                 1            -47                   1   
5       1029                 1            -46                   1   
6       3604                 1            -44                   1   
7       5671                 1            -43                   1   
8       2300                 1            -43                   1   
9       4074                 1            -42                   1   

           4                  5                         6    \
0  caps_intake  pcl5_score_intake  pcl5month_score.baseline   
1           15                 71                        75   
2           23                 72                  

### Q2.2. Multiple Linear Regression (7')

<font size='4'>

- Select columns specified in the following code chunk and create a subset dataset named `ptsd_sub_df`.
- Fit a linear regression to examine the association between `caps_intake` (continuous outcome) and the remaining covariates (as predictors) using `ptsd_sub_df`.
    - Note that all covariates ending with `_code` are categorical variables.
- Use the instruction here to write the formula for linear regression in Python.
    - https://www.statsmodels.org/stable/example_formulas.html
- Report the output page including R2, adjusted R2, and parameter estimates, SE, 95% confidence intervals, and p-values.
- Provide a brief interpretation for all significant predictors (p<0.05) excluding the intercept.
- Relevant label information includes:
    - `employment_code`: 1: Employed, 2: Unemployed, 3: Retired, 4: Disabled/Unable to work, 5: Student, 6: Other.
    - `rank_code`: 1. Enlisted, 2: Officer, 3: Other

In [26]:
# The following column names are used for linear regression.
# Do not delete.
relevant_col_names = ['caps_intake', 'age_iop', 'gender_code', 'sexualorient_code', 'race_code', 'ethnicity_code', 
                      'education_code', 'employment_code',
                      'rank_code', 'branch_code', 'mdd_code', 'ctq_total_score', 'sexual_trauma', 'sud_code']

In [30]:
# Write your own code
cols_to_use = [
    'caps_intake',
    'pcl5_score_intake',
    'pcl5month_score.3_month_follow_up',
    'mdd_code',
    'employment_code',
    'rank_code'
]

ptsd_sub_df = ptsd_df[cols_to_use].copy()

# Rename columns to remove dots for Patsy formula
ptsd_sub_df.rename(columns={
    'pcl5month_score.3_month_follow_up': 'pcl5month_score_followup'
}, inplace=True)

# Convert categorical variables to 'category' dtype
categorical_cols = [col for col in ptsd_sub_df.columns if col.endswith('_code')]
for col in categorical_cols:
    ptsd_sub_df[col] = ptsd_sub_df[col].astype('category')

# Ensure numeric columns are numeric
ptsd_sub_df['caps_intake'] = pd.to_numeric(ptsd_sub_df['caps_intake'], errors='coerce')
ptsd_sub_df['pcl5_score_intake'] = pd.to_numeric(ptsd_sub_df['pcl5_score_intake'], errors='coerce')
ptsd_sub_df['pcl5month_score_followup'] = pd.to_numeric(ptsd_sub_df['pcl5month_score_followup'], errors='coerce')

# ----------------------------
# Step 2: Fit linear regression
# ----------------------------
formula = 'caps_intake ~ pcl5_score_intake + pcl5month_score_followup + C(mdd_code) + C(employment_code) + C(rank_code)'
model = smf.ols(formula=formula, data=ptsd_sub_df).fit()

# ----------------------------
# Step 3: Output results
# ----------------------------
print(f"R-squared: {model.rsquared:.3f}")
print(f"Adjusted R-squared: {model.rsquared_adj:.3f}\n")

# Regression table with coefficients, SE, 95% CI, and p-values
results_table = model.summary2().tables[1]
print(results_table)

# ----------------------------
# Step 4: Highlight significant predictors (p < 0.05)
# ----------------------------
significant = results_table[results_table['P>|t|'] < 0.05].drop('Intercept', errors='ignore')
print("\nSignificant predictors (p < 0.05):")
print(significant)

# ----------------------------
# Step 5: Interpretation guide
# ----------------------------
for idx, row in significant.iterrows():
    coef = row['Coef.']
    pval = row['P>|t|']
    if idx.startswith('C('):  # categorical variable
        cat_name = idx.split('[')[0].replace('C(', '').replace(')', '')
        level = idx.split('[')[1].replace(']', '')
        print(f"- {cat_name} level {level} is associated with a change of {coef:.2f} in caps_intake (p={pval:.3f}) compared to the reference category.")
    else:  # continuous variable
        print(f"- {idx} increases caps_intake by {coef:.2f} units per 1 unit increase (p={pval:.3f}).")

# Significant predictors (p < 0.05)
significant = results_table[results_table['P>|t|'] < 0.05].drop('Intercept', errors='ignore')
print("\nSignificant predictors (p < 0.05):")
print(significant)

R-squared: 0.249
Adjusted R-squared: 0.215

                              Coef.  Std.Err.          t         P>|t|  \
Intercept                 24.612507  2.094915  11.748688  4.435484e-25   
C(mdd_code)[T.1]           0.080726  1.159766   0.069605  9.445707e-01   
C(employment_code)[T.2]   -2.300135  1.393447  -1.650680  1.002244e-01   
C(employment_code)[T.3]   -1.695277  1.616608  -1.048663  2.954787e-01   
C(employment_code)[T.4]    1.097661  1.501160   0.731209  4.654263e-01   
C(employment_code)[T.5]   -4.905105  2.259130  -2.171236  3.097829e-02   
C(employment_code)[T.6]   -0.711157  3.156494  -0.225300  8.219544e-01   
C(rank_code)[T.2]         -3.398522  1.504843  -2.258389  2.489776e-02   
C(rank_code)[T.3]         -6.055097  3.527361  -1.716608  8.745191e-02   
pcl5_score_intake          0.231904  0.039700   5.841374  1.836813e-08   
pcl5month_score_followup   0.003399  0.029991   0.113350  9.098563e-01   

                             [0.025     0.975]  
Intercept         

In [31]:
# Write your interpretations below:
#The linear regression model explained approximately 24.9% of the variance in caps_intake (R² = 0.249), with an adjusted R² of 0.215 after accounting for the number of predictors. Among the covariates, three predictors were statistically significant (p < 0.05). Baseline PCL5 scores (pcl5_score_intake) were positively associated with caps_intake, such that for every 1-unit increase in the baseline score, caps_intake increased by 0.23 units, holding all other variables constant. Regarding categorical predictors, participants who were students (employment_code level 5) had, on average, 4.91 units lower caps_intake compared to employed participants, controlling for other factors. Additionally, participants with rank level 2 (Officers) had, on average, 3.40 units lower caps_intake compared to enlisted personnel, holding other variables constant. Other predictors, including follow-up PCL5 scores and the remaining employment and rank levels, were not statistically significant in predicting caps_intake.
